In [2]:
import wandb; wandb.login()
from transformers import RobertaForMaskedLM, RobertaConfig, GPT2TokenizerFast

config = RobertaConfig(

    # number of tokens in the vocabulary 
    vocab_size = 10_000, 
    # embedding size (vector length) of each token 
    hidden_size=512, 
    # we thus have an embedding block of 512 x 10'000 parameters

    # maximum sequence length, though inputs longer than `hidden_size` will be iteratively processed
    max_position_embeddings = 512, 

    # number of transformer blocks. div by 2 for attention_types
    num_layers=2, 
    # for global and local attention (GPT-Neo-specific)
    # attention_types=[[["global", "local"], 1]], 

    num_heads=4,     # attention heads
    window_size=256, # for local attention (GPT-Neo-specific)

    intermediate_size=1024, # size of 'up-projection' layer in FFN
)
# config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ar4l. Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import RobertaTokenizer
# tokenizer = GPT2TokenizerFast.from_pretrained('10k-gpt-neo', model_max_length=config.hidden_size)
# tokenizer = RobertaTokenizer.from_pretrained('10k-gpt-neo', model_max_length=config.hidden_size)

assert tokenizer.model_max_length == 512 
assert tokenizer.vocab_size == config.vocab_size

print(f'padding token is {tokenizer.pad_token}')
# HF wasn't saving this nor the tokenizer's pad_token

print(f'padding token in config: {config.pad_token_id}, in tokeniser: {tokenizer.pad_token_id}')
config.pad_token_id = tokenizer.pad_token_id

padding token is <|endoftext|>
padding token in config: 1, in tokeniser: 9999


#### Creating a `RoBERTa` TinyStories Tokeniser
Akin to `pre_train.ipynb`, we create a new tokeniser with 10k token vocabulary. But, this time we maintain the `BERT` special tokens, as they are a superset of the `GPT-Neo` special tokens.

In [7]:
tok = RobertaTokenizer.from_pretrained('FacebookAI/roberta-base')
print(f'we want 10,000 tokens instead of {tok.vocab_size:,}')

tok.save_pretrained('10k-roberta-base')
# NOTE: Manually remove 40,265 entries from `merges.txt` and `vocab.json`

we want 10,000 tokens instead of 50,265


('10k-roberta-base/tokenizer_config.json',
 '10k-roberta-base/special_tokens_map.json',
 '10k-roberta-base/vocab.json',
 '10k-roberta-base/merges.txt',
 '10k-roberta-base/added_tokens.json')

In [9]:
model = RobertaForMaskedLM(config)
print(f'The model has {model.num_parameters():,} parameters.')

AssertionError: Padding_idx must be within num_embeddings

In [13]:
config.pad_token_id

9999

In [14]:
config.max_position_embeddings

512